In [1]:
# Markov Model Chain Python Example
# https://www.datacamp.com/community/tutorials/markov-chains-python-tutorial

In [2]:
import numpy as np
import random as rm

In [3]:
# The statespace
states = ["Sleep","Icecream","Run"]

# Possible sequences of events
transitionName = [["SS","SR","SI"],["RS","RR","RI"],["IS","IR","II"]]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.2,0.6,0.2],[0.1,0.6,0.3],[0.2,0.7,0.1]]

In [4]:
if sum(transitionMatrix[0]) + sum(transitionMatrix[1]) + sum(transitionMatrix[1]) != 3:
    print("Somewhere, something went wrong. Transition matrix, perhaps?")
else: 
    print("All is gonna be okay, you should move on!! ;)")    

All is gonna be okay, you should move on!! ;)


In [5]:
# A function that implements the Markov model to forecast the state/mood. 
def activity_forecast(days):
    # Choose the starting state
    activityToday = "Sleep"
    print("Start state: " + activityToday)
    # Shall store the sequence of states taken. So, this only has the starting state for now.
    activityList = [activityToday]
    i = 0
    # To calculate the probability of the activityList
    prob = 1
    while i != days:
        if activityToday == "Sleep":
            change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix[0])
            if change == "SS":
                prob = prob * 0.2
                activityList.append("Sleep")
                pass
            elif change == "SR":
                prob = prob * 0.6
                activityToday = "Run"
                activityList.append("Run")
            else:
                prob = prob * 0.2
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Run":
            change = np.random.choice(transitionName[1],replace=True,p=transitionMatrix[1])
            if change == "RR":
                prob = prob * 0.5
                activityList.append("Run")
                pass
            elif change == "RS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.3
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Icecream":
            change = np.random.choice(transitionName[2],replace=True,p=transitionMatrix[2])
            if change == "II":
                prob = prob * 0.1
                activityList.append("Icecream")
                pass
            elif change == "IS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else: 
                prob = prob * 0.7
                activityToday = "Run"
                activityList.append("Run")
        i += 1  
    print("Possible states: " + str(activityList))
    print("End state after "+ str(days) + " days: " + activityToday)
    print("Probability of the possible sequence of states: " + str(prob))

# Function that forecasts the possible state for the next 2 days
activity_forecast(2)

Start state: Sleep
Possible states: ['Sleep', 'Run', 'Icecream']
End state after 2 days: Icecream
Probability of the possible sequence of states: 0.18


# Sherlock Holmes Example ussing Markov Chains

In [9]:
import os
import re
import nltk
import string
import random
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/franciscofurey/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
# Define the path to the directory containing the stories.
story_path = './data/sherlock/sherlock/'

def read_all_stories(story_path: str) -> list:
    """
    Reads all text files from a specified directory, stopping at a delimiter or an empty line.
    
    Parameters:
        story_path (str): The path to the directory containing story files.
    
    Returns:
        list: A list of lines from all files, excluding delimiter lines and empty lines.
    """
    lines = []
    for _, _, files in os.walk(story_path):
        for file in files:
            with open(os.path.join(story_path, file)) as f:
                for line in f:
                    line = line.strip()
                    if line == '----------': break
                    if line:
                        lines.append(line)
    return lines
        
stories = read_all_stories(story_path=story_path)
print("Number of lines = ", len(stories))

def clean_txt(txt: list) -> list:
    """
    Cleans a list of text lines by converting to lowercase, removing punctuation, and tokenizing.
    
    Parameters:
        txt (list): A list of text lines to be cleaned.
    
    Returns:
        list: A list of cleaned and tokenized words.
    """
    cleaned_words = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\\[\]]", "", line)
        tokens = word_tokenize(line)
        # Filter out tokens that are not alphabetic
        words = [word for word in tokens if word.isalpha()]
        cleaned_words += words
    return cleaned_words

cleaned_stories = clean_txt(txt=stories)
print("Number of words = ", len(cleaned_stories))


Number of lines =  215021
Number of words =  2332247


In [22]:
def make_markov_model(cleaned_stories: list, n_gram: int = 2) -> dict:
    """
    Constructs a Markov model from a list of words.

    This function creates a dictionary representing the transitions between n-gram states
    to the next possible states along with their probabilities.

    Parameters:
        cleaned_stories (list): A list of cleaned and tokenized words from stories.
        n_gram (int): The number of words in the state used for the Markov model.

    Returns:
        dict: A dictionary representing the Markov model where keys are current states
              and values are dictionaries of next possible states and their probabilities.
    """
    markov_model = {}  # Initialize an empty dictionary for the Markov model

    # Loop through the list of words to populate the Markov model
    for i in range(len(cleaned_stories) - n_gram - 1):
        curr_state, next_state = "", ""
        # Construct the current and next state by concatenating words
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]  # Remove the trailing space
        next_state = next_state[:-1]

        # If the current state is not in the model, add it
        if curr_state not in markov_model:
            markov_model[curr_state] = {next_state: 1}
        else:
            # If the next state exists, increment; otherwise, add it with a count of 1
            markov_model[curr_state].setdefault(next_state, 0)
            markov_model[curr_state][next_state] += 1
    
    # Calculate transition probabilities for each state
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count / total  # Convert counts to probabilities
        
    return markov_model

# Example usage
markov_model = make_markov_model(cleaned_stories=cleaned_stories)
print("Number of states = ", len(markov_model.keys()))
print("All possible transitions from 'the game' state: \n")
if 'the game' in markov_model:
    print(markov_model['the game'])
else:
    print("The state 'the game' does not exist in the model.")

Number of states =  208717
All possible transitions from 'the game' state: 

{'is up': 0.06306306306306306, 'is and': 0.036036036036036036, 'was afoot': 0.036036036036036036, 'for the': 0.036036036036036036, 'was whist': 0.036036036036036036, 'would have': 0.036036036036036036, 'in their': 0.036036036036036036, 'was up': 0.09009009009009009, 'in that': 0.036036036036036036, 'the lack': 0.036036036036036036, 'for all': 0.06306306306306306, 'is afoot': 0.036036036036036036, 'was in': 0.02702702702702703, 'is hardly': 0.02702702702702703, 'may wander': 0.02702702702702703, 'now a': 0.02702702702702703, 'my own': 0.02702702702702703, 'at any': 0.02702702702702703, 'mr holmes': 0.02702702702702703, 'ay whats': 0.02702702702702703, 'my friend': 0.02702702702702703, 'fairly by': 0.02702702702702703, 'is not': 0.02702702702702703, 'was not': 0.02702702702702703, 'worth it': 0.02702702702702703, 'you are': 0.02702702702702703, 'i am': 0.02702702702702703, 'now count': 0.02702702702702703, 'your

In [27]:
def generate_story(markov_model: dict, limit: int = 100, start: str = 'my god') -> str:
    """
    Generates a story based on a given Markov model, starting from a specified state.

    Parameters:
        markov_model (dict): The Markov model to use for generating the story.
        limit (int): The maximum number of words in the generated story.
        start (str): The starting state (words) for the story.

    Returns:
        str: A string representing the generated story.
    """
    n = 0  # Initialize word counter
    curr_state = start  # Set the current state to the starting words
    story = curr_state + " "  # Initialize the story with the starting state

    # Generate the story up to the word limit
    while n < limit and curr_state in markov_model:
        # Choose the next state based on the distribution in the Markov model
        next_state = random.choices(
            population=list(markov_model[curr_state].keys()),
            weights=list(markov_model[curr_state].values()),
            k=1  # Choose one next state
        )[0]

        story += next_state + " "  # Append the next state to the story
        curr_state = next_state  # Update the current state to the next state
        n += 1  # Increment word counter

    return story.strip()  # Return the story, removing any trailing space

# Example usage: Generate and print 20 short stories
for i in range(20):
    story = generate_story(markov_model=markov_model, start="dear holmes", limit=8)
    print(f"{i}. {story}")

print("-------------------")
print("A longer story:")
print(generate_story(markov_model, start="the case", limit=100))


0. dear holmes if i had known ralph smith who went to the mystery in the matter as a
1. dear holmes i have it here in my heart for the test with me now and so atone
2. dear holmes am i he gazed from sir charles for several days in bed and we may now
3. dear holmes am i to do the theorizing what sort of proposal is that i never wanted anything
4. dear holmes i thought it was clear enough against you for this one excursion he spent his days
5. dear holmes that i am still at a small wayside station and it has sworn to his identity
6. dear holmes i thought he was telling them that there can be handed out in an instant and
7. dear holmes am i a wandering american with a wonderful worker if i pay him well certainly that
8. dear holmes he has been quite long enough for me to execute this said baynes what do you
9. dear holmes my previous letters and supposing that this unhappy lady to whom he observed if i remember
10. dear holmes said i nodding towards my house i wont listen to this you can 